# CATE Algorithm Evaluation with Synthetic Data

This notebook 

1. shows how to use EBM and econml to generate EBM CATE estimators. 
2. setup a complete performance evaluation/comparison pipeline for any CATE estimators. 


[To be added: some visualization]



### DGP output
```
data = {'X':, 'P':,'T':, 'Y':,'Y0':, 'Y1':, 
        'treatment_effect':, 'X_test':, 'treatment_effect_test':,'Y_test':,'special_test':}
```
The generated data is formated as valid input of BaseCateEstimator.fit() in the econml package:
```
    Y: (n × d_y) matrix of outcomes for each sample
    T: (n × d_t) matrix of treatments for each sample
    X: optional (n × d_x) matrix of features for each sample
    W: optional (n × d_w) matrix of controls for each sample
    Z: optional (n × d_z) matrix of instruments for each sample
    inference: optional string, `Inference` instance, or None
        Method for performing inference.  All estimators support 'bootstrap'
        (or an instance of `BootstrapInference`), some support other methods as well.
```

### 

### Evaluation Criteria
MSE of tau: $MSE(\tau, \hat{\tau}) = \frac{1}{n}\sum_{i=1}^n (\hat{\tau}(x_i) - \tau_i)^2$

## util functions
- `data_generator`: a general DGP generator 
- `regressor`: an automl regressor based on GridSearchCV
- `clf`:  an automl classifier based on GridSearchCV
- `generate_EBM_CATE_learners`: generate a dictionary of EBM CATE learners
- `generate_CATE_learners`: generate a dictionary of CATE learners
- `run_and_report_mse`: report mse of estimating tau for a list of models

In [1]:
from CATE_utils import data_generator, regressor, clf, generate_EBM_CATE_learners, generate_CATE_learners, run_and_report_mse
import numpy as np
import scipy
import pandas as pd
import time
from econml.dr import DRLearner, LinearDRLearner, SparseLinearDRLearner, ForestDRLearner
from econml.metalearners import TLearner, SLearner, XLearner, DomainAdaptationLearner
from econml.orf import DROrthoForest
from econml.dml import DML, LinearDML, SparseLinearDML, CausalForestDML, NonParamDML
from econml.sklearn_extensions.model_selection import GridSearchCVList
from econml.sklearn_extensions.linear_model import WeightedLassoCVWrapper, WeightedLasso, WeightedLassoCV
from sklearn.base import clone
from sklearn.preprocessing import PolynomialFeatures
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LinearRegression, LassoCV, Lasso, LogisticRegression
from econml.sklearn_extensions.linear_model import WeightedLassoCV
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier, RandomForestRegressor, RandomForestClassifier
from interpret.glassbox import ExplainableBoostingRegressor, ExplainableBoostingClassifier
import lightgbm as lgb

from interpret import show
from matplotlib import pyplot as plt

## DGP 1 
(From 'Doubly Robust Learner an Interpretability.ipynb')
* Binary covariates (segments naturally defined)
* $X_{i1}$ determines both $p$ and $\tau$, $X_{i1}$ is the only confounder and generates heterogeneity
* Heteroskedastic error in $X_{i1}$
* $X_{i-1}$ are irrelevant for $p$ or $\tau$ or heteroskedasticity

\begin{align}
X_i & = (X_{i1}, X_{i2}, X_{i3}, X_{i4}) \overset{iid}{\sim} \operatorname{Ber}(0.5)\\
p_i & = \operatorname{expit}(X_{i1}) = \left\{
\begin{array}{cc}
0.5 & X_{i1} = 0\\
0.73 & X_{i1} = 1
\end{array}\right.\\
Y_i(0) & = X_{i1} + (X_{i1} + 1)*\operatorname{N}(0, 1)\\
\tau_i & = -1 + 2 X_{i1}
\end{align}

## DGP 2 
(Frome Metalearner Examples.ipynb)
We use the data generating process (DGP) from [Kunzel et al.](https://arxiv.org/abs/1706.03461). The DGP is described by the following equations:

\begin{align}
X_i & \sim N(0, I)\\
p_i & =  \left\{
\begin{array}{cc}
0.8 & X_{i3} \in [-0.5, 0.5]\\
0.2 & X_{i3} \notin [-0.5, 0.5]
\end{array}\right.\\
Y_i(0) & = X_i^T\beta + \operatorname{N}(0, 1)\\
\tau_i & = 8 \mathbb{I}(x_2>0.1)
\end{align}


## DGP3
We use the Response Surface B from [Hill (2011)](https://www.tandfonline.com/doi/pdf/10.1198/jcgs.2010.08162) to generate sythetic outcome surfaces from real-world covariates and treatment assignments (Infant Health Development Program data). Since the original data was part of a randomized trial, a subset of the treated infants (those with non-white mothers) has been removed from the data in order to mimic the observational data setting. For more details, see [Hill (2011)](https://www.tandfonline.com/doi/pdf/10.1198/jcgs.2010.08162).


The DGP is described by the following equations:

$
Y(0) = e^{(X+W)\beta} + \epsilon_0, \;\epsilon_0 \sim N(0, 1)\\
Y(1) = X\beta - \omega + \epsilon_1, \;\epsilon_1 \sim N(0, 1)\\
$

where $X$ is a covariate matrix, $W$ is a constant matrix with entries equal to $0.5$ and $w$ is a constant calculated such that the CATT equals $4$.


## DGP4 
(From 'Double Machine Learning Examples.ipynb')
We use the data generating process (DGP) from [Oprescu 2019](https://arxiv.org/abs/1806.03467). The DGP is described by the following equations:

\begin{align}
X_i \sim& \text{Uniform}(0,1)\\
W_i \sim& \text{Normal}(0,\, I_{30})\\
p_i =& expit(W_i^T\beta)\\
Y_i(0) =& \langle W, \gamma\rangle + \epsilon, &\; \epsilon \sim \text{Uniform}(-1, 1)\\
\tau_i =& \exp(2\cdot x_1) 
\end{align}

where $W$ is a matrix of high-dimensional confounders and $\beta, \gamma$ have high sparsity.


In [2]:
# # Treatment effect function
# def exp_te(x):
#     return np.exp(2 * x[0])# DGP constants

# np.random.seed(123)
# n = 1000
# n_w = 30
# support_size = 5
# n_x = 4
# # Outcome support
# support_Y = np.random.choice(range(n_w), size=support_size, replace=False)
# coefs_Y = np.random.uniform(0, 1, size=support_size)
# epsilon_sample = lambda n:np.random.uniform(-1, 1, size=n)
# # Treatment support
# support_T = support_Y
# coefs_T = np.random.uniform(0, 1, size=support_size)
# eta_sample = lambda n: np.random.uniform(-1, 1, size=n) 

# # Generate controls, covariates, treatments and outcomes
# W = np.random.normal(0, 1, size=(n, n_w))
# X = np.random.uniform(0, 1, size=(n, n_x))
# # Heterogeneous treatment effects
# TE = np.array([exp_te(x_i) for x_i in X])
# # Define treatment
# log_odds = np.dot(W[:, support_T], coefs_T) + eta_sample(n)
# T_sigmoid = 1/(1 + np.exp(-log_odds))
# T = np.array([np.random.binomial(1, p) for p in T_sigmoid])
# # Define the outcome
# Y = TE * T + np.dot(W[:, support_Y], coefs_Y) + epsilon_sample(n)

# # get testing data
# X_test = np.random.uniform(0, 1, size=(n, n_x))
# X_test[:, 0] = np.linspace(0, 1, n)
# expected_te = X_test

In [3]:
# # DGP3
# n = 1000
# d = 5
# covariates_model = lambda d: multivariate_normal(np.zeros(d), np.diag(np.ones(d)), 1)
# propensity_model = lambda x: (0.8 if (x[2]>-0.5 and x[2]<0.5) else 0.2)
# beta = np.random.uniform(-3, 3, d)
# control_outcome_model = lambda x: np.dot(x, beta) + np.random.normal(0, 1)
# treatment_effect_model = lambda x: (1 if x[1] > 0.1 else 0)*8
# data2 = data_generator(n=1000, 
#                       d=4, 
#                       covariates_model=covariates_model,
#                       propensity_model=propensity_model,
#                       control_outcome_model=control_outcome_model,
#                       treatment_effect_model=treatment_effect_model,
#                       seed = 1234
#                       )

In [4]:
# # DGP2
# n = 1000
# d = 5
# covariates_model = lambda d: multivariate_normal(np.zeros(d), np.diag(np.ones(d)), 1)
# propensity_model = lambda x: (0.8 if (x[2]>-0.5 and x[2]<0.5) else 0.2)
# beta = np.random.uniform(-3, 3, d)
# control_outcome_model = lambda x: np.dot(x, beta) + np.random.normal(0, 1)
# treatment_effect_model = lambda x: (1 if x[1] > 0.1 else 0)*8
# data2 = data_generator(n=1000, 
#                       d=4, 
#                       covariates_model=covariates_model,
#                       propensity_model=propensity_model,
#                       control_outcome_model=control_outcome_model,
#                       treatment_effect_model=treatment_effect_model,
#                       seed = 1234
#                       )


In [5]:
# DGP1
n = 200
d = 4
covariates_model = lambda d: np.random.binomial(1, .5, size=d)
propensity_model = lambda x: scipy.special.expit(x[0])
control_outcome_model = lambda x: x[0] + (x[0] + 1)*np.random.normal(0, 1, size=1)
treatment_effect_model = lambda x: -1 + 2 * x[0]
special_test_point = np.array([[1, 0, 0, 0], [0, 0, 0, 0]])
special_test_value = np.array([1, -1])


data1 = data_generator(n=1000, 
                      d=4, 
                      covariates_model=covariates_model,
                      propensity_model=propensity_model,
                      control_outcome_model=control_outcome_model,
                      treatment_effect_model=treatment_effect_model,
                      special_test_point=special_test_point,
                      special_test_value=special_test_value,
                      seed = 1234
                      )

## Read the data

In [6]:
data = data1
X = data['X']
T = data['T']
Y = data['Y']
X_test = data['X_test']
treatment_effect_test = data['treatment_effect_test']
Y_test = data['Y_test']
# P = data['P']
# Y0 = data['Y0']
# Y1 = data['Y1']
# treatment_effect = data['treatment_effect']
# special_test_point, special_test_value = data['special_test']

model_y = clone(regressor().fit(X, Y).best_estimator_)
model_t = clone(clf().fit(X, T).best_estimator_)

n = X.shape[0]

## Train the Estimator

In [13]:
seed =100
est_NonParamDML = NonParamDML(model_y=ExplainableBoostingRegressor(random_state=seed, outer_bags=20),
                                model_t=model_t,
                                model_final=ExplainableBoostingRegressor(random_state=seed, outer_bags=20),
                                discrete_treatment=True,
                                random_state=seed)
est_NonParamDML.fit(Y, T, X=X)

In [14]:
show(est_NonParamDML.model_cate.explain_global())

<!-- http://127.0.0.1:7724/140662504437136/ -->

In [9]:
learner_dic = generate_CATE_learners(model_y, model_t, X)
ebm_learner_dic = generate_EBM_CATE_learners(model_y, model_t)

records = run_and_report_mse(['Tlearner', 'NonParamDML'], learner_dic, Y, T, X, X_test, treatment_effect_test) \
    + run_and_report_mse(['EBM_Tlearner', 'EBM_NonParamDML'], ebm_learner_dic, Y, T, X, X_test, treatment_effect_test)
# records = run_and_report_mse(learner_dic.keys(), learner_dic, Y, T, X, X_test, treatment_effect_test) + run_and_report_mse(ebm_learner_dic.keys(), ebm_learner_dic, Y, T, X, X_test, treatment_effect_test)


Tlearner
NonParamDML


One or more of the test scores are non-finite: [nan]


TypeError: fit() got an unexpected keyword argument 'sample_weight'

## Report Result

In [ ]:
record_df = pd.DataFrame.from_records(records)[['model_name', 'mse', 'error_50', 'error_95']]

In [ ]:
record_df.sort_values(by=['mse'])

,model_name,mse,error_50,error_95
3,EBM_NonParamDML,1.779549,0.517346,2.092989
1,NonParamDML,1.945719,0.292297,2.189975
0,Tlearner,1.983921,0.465572,2.225324
2,EBM_Tlearner,1.992881,0.296013,2.243950


## EBM CATE visualization

In [ ]:
show(ebm_learner_dic['EBM_NonParamDML'].model_cate.explain_global())

<!-- http://127.0.0.1:7001/139882456186384/ -->